In [1]:

def calc_level_users(user, max_level, find_next_level):
    next_queue = [user]
    queue = []
    ret = []
    for _ in xrange(0, max_level):
        queue = next_queue
        next_queue = []
        for q in queue:
            next_queue += find_next_level(q)
        ret.append(next_queue[:])
    return ret

In [8]:
data = [
    [1, 2],
    [1, 3],
    [2, 4],
    [2, 5],
    [3, 6],
    [3, 7],
    [4, 8],
    [4, 9],
    [5, 10],
    [10, 11],
]

In [6]:

def find_next_level(q):
    global data
    return [y[1] for y in filter(lambda x: x[0] == q, data)]

In [9]:
calc_level_users(1, 3, find_next_level)

[[2, 3], [4, 5, 6, 7], [8, 9, 10]]